<a href="https://colab.research.google.com/github/tcreamype3R/Colab-Publicar/blob/main/Metodo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def extract_linkedin_data_standardized(text, period):
    """
    Función estandarizada para extraer datos de cualquier período
    """
    # Patrón único para todas las extracciones
    pattern = r'(\*{2}Bernabe Aguirre Carrasco\*{2}.*?)(?=\*{2}Bernabe Aguirre Carrasco\*{2}|$)'
    publications = re.findall(pattern, text, re.DOTALL)

    # Métricas consistentes para todos los períodos
    standardized_data = []
    for i, pub in enumerate(publications):
        publication_data = {
            'periodo': period,
            'numero': i + 1,
            'impresiones': extract_metric(pub, r'\[(\d+)\]'),
            'comentarios': extract_metric(pub, r'(\d+)\s*comentarios?'),
            'compartidos': extract_metric(pub, r'(\d+)\s*veces?\s*compartido'),
            'reacciones': extract_metric(pub, r'(\d+)\s*reacciones?'),
            'hashtags': extract_hashtags(pub),
            'texto': extract_text(pub),
            'fecha': extract_date(pub),
            'tipo_contenido': classify_content_type(pub)
        }
        standardized_data.append(publication_data)

    return standardized_data

In [4]:
STANDARD_METRICS = {
    'impresiones': {'tipo': 'absoluto', 'unidad': 'número'},
    'comentarios': {'tipo': 'absoluto', 'unidad': 'número'},
    'compartidos': {'tipo': 'absoluto', 'unidad': 'número'},
    'reacciones': {'tipo': 'absoluto', 'unidad': 'número'},
    'engagement_total': {'tipo': 'calculado', 'formula': 'comentarios + compartidos + reacciones'},
    'tasa_engagement': {'tipo': 'porcentaje', 'formula': '(engagement_total / impresiones) * 100'},
    'alcance_promedio': {'tipo': 'promedio', 'formula': 'mean(impresiones)'}
}

In [6]:
def validate_data_coverage(publications, expected_count, period):
    """
    Valida que tengamos suficiente cobertura de datos
    """
    coverage_ratio = len(publications) / expected_count
    if coverage_ratio < 0.8:  # Mínimo 80% de cobertura
        raise ValueError(f"Cobertura insuficiente para {period}: {coverage_ratio:.1%}")
    return coverage_ratio

In [8]:
COMPARATIVE_METRICS = {
    'engagement_rate_promedio': 'mean(tasa_engagement)',
    'crecimiento_impresiones': '(impresiones_periodo_actual - impresiones_periodo_anterior) / impresiones_periodo_anterior * 100',
    'consistencia_publicacion': 'std(impresiones) / mean(impresiones)'
}

In [9]:
def validate_data_coverage(publications, expected_count, period):
    """
    Valida que tengamos suficiente cobertura de datos
    """
    coverage_ratio = len(publications) / expected_count
    if coverage_ratio < 0.8:  # Mínimo 80% de cobertura
        raise ValueError(f"Cobertura insuficiente para {period}: {coverage_ratio:.1%}")
    return coverage_ratio

In [10]:
def process_all_periods(periods_data):
    """
    Procesa todos los períodos con la misma metodología
    """
    results = {}
    for period, data in periods_data.items():
        # Aplicar las mismas transformaciones a todos los períodos
        df = pd.DataFrame(data)
        df = calculate_standard_metrics(df)
        df = classify_content(df)
        df = extract_peak_performance(df)

        results[period] = {
            'dataframe': df,
            'summary': generate_summary_stats(df, period),
            'coverage': len(df) / get_expected_count(period)
        }

    return results

In [11]:
def comparative_analysis(results):
    """
    Análisis comparativo entre períodos
    """
    comparison_data = []

    for period, result in results.items():
        df = result['dataframe']
        summary = {
            'periodo': period,
            'total_publicaciones': len(df),
            'impresiones_totales': df['impresiones'].sum(),
            'engagement_promedio': df['engagement_total'].mean(),
            'tasa_engagement_promedio': df['tasa_engagement'].mean(),
            'mejor_publicacion': df.loc[df['engagement_total'].idxmax()]['texto'][:100] + "...",
            'cobertura': result['coverage']
        }
        comparison_data.append(summary)

    return pd.DataFrame(comparison_data)

In [12]:
def calculate_trend_metrics(comparison_df):
    """
    Calcula métricas de tendencia entre períodos
    """
    trends = []
    periods = comparison_df['periodo'].tolist()

    for i in range(1, len(periods)):
        current = comparison_df.iloc[i]
        previous = comparison_df.iloc[i-1]

        trend = {
            'periodo': f"{previous['periodo']} → {current['periodo']}",
            'crecimiento_impresiones': ((current['impresiones_totales'] - previous['impresiones_totales']) / previous['impresiones_totales']) * 100,
            'cambio_engagement': current['tasa_engagement_promedio'] - previous['tasa_engagement_promedio'],
            'cambio_consistencia': current['cobertura'] - previous['cobertura']
        }
        trends.append(trend)

    return pd.DataFrame(trends)

In [13]:
def create_comparative_visualization(comparison_df):
    """
    Crea visualizaciones comparativas consistentes
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))

    # Impresiones totales por período
    axes[0, 0].bar(comparison_df['periodo'], comparison_df['impresiones_totales'])
    axes[0, 0].set_title('Impresiones Totales por Período')

    # Tasa de engagement comparativa
    axes[0, 1].plot(comparison_df['periodo'], comparison_df['tasa_engagement_promedio'], 'o-')
    axes[0, 1].set_title('Tasa de Engagement Promedio')

    # Distribución de engagement
    engagement_data = [results[period]['dataframe']['engagement_total'] for period in comparison_df['periodo']]
    axes[1, 0].boxplot(engagement_data, labels=comparison_df['periodo'])
    axes[1, 0].set_title('Distribución de Engagement')

    # Cobertura de datos
    axes[1, 1].bar(comparison_df['periodo'], comparison_df['cobertura'] * 100)
    axes[1, 1].set_title('Cobertura de Datos (%)')

    plt.tight_layout()
    return fig

In [15]:
def run_complete_analysis():
    """
    Ejecuta el análisis completo y estandarizado
    """
    periods = {
        '7_dias': load_data('7_dias.docx'),
        '14_dias': load_data('14_dias.docx'),
        '28_dias': load_data('28_dias.docx'),
        '90_dias': load_data('90_dias.docx')
    }

    # Validar y procesar todos los períodos
    validated_data = {}
    for period_name, data in periods.items():
        coverage = validate_data_coverage(data, EXPECTED_COUNTS[period_name], period_name)
        processed_data = process_period_data(data, period_name)
        validated_data[period_name] = {
            'data': processed_data,
            'coverage': coverage
        }

    # Análisis comparativo
    comparative_results = comparative_analysis(validated_data)
    trends = calculate_trend_metrics(comparative_results)

    # Generar reporte
    generate_report(validated_data, comparative_results, trends)